In [ ]:
#using Pkg
#Pkg.add("DifferentialEquations");
#Pkg.add("StatsBase");

## Part 1: systems with more than one species

## 1. System 1

$\emptyset \xrightarrow{\kappa_1}\mathcal{A}\xrightarrow{\kappa_2}\emptyset$

$\mathcal{A}\xrightarrow{\kappa_3}\mathcal{B}\xrightarrow{\kappa_4}\emptyset $

#### Define a function that computes the RHS of the chemical master equation.
#### You are free to choose whether to implement it in a vector or in a matrix form (e.g. rows for number of $\mathcal{A}$ molecules, columns for number of $\mathcal{B}$ molecules)

In [ ]:
using DifferentialEquations

In [ ]:
n_upperbound = 30

## If im matrix form:
Avec = collect(0:n_upperbound)
Bvec = collect(0:n_upperbound)

p = Iterators.product(Avec, Bvec)
points = collect.(p)
points_reshaped = reduce(hcat,points)
Apoints_matrix = reshape(points_reshaped[1,:], n_upperbound+1, n_upperbound+1)
Bpoints_matrix = reshape(points_reshaped[2,:], n_upperbound+1, n_upperbound+1)

function AB_system!(du, u, p, t)
    
    κ₁, κ₂, κ₃, κ₄, V = p

    u ./= sum(u) 
    
    ####
    #### Padding here (if necessary)
    ####
    
    A = (-κ₂*####
         -κ₃*####
         -κ₄*####
         -κ₁*####)
    B = κ₁*####   
    C = κ₂*####
    D = κ₃*####
    E = κ₄*####
        
    du .= #### RHS here
   
end

#### Set the initial condition, the time interval of interest and the parameters.

In [ ]:
u₀ = zeros(####, ####)
u₀[1, 1] = ####

T = 20.0
tspan = ####

κ₁, κ₂, κ₃, κ₄, V = ####
p = [κ₁, κ₂, κ₃, κ₄, V];

#### Create an ordinary differential equation problem with the variables defined above.

In [ ]:
prob = ####

#### Solve the ODE problem

In [ ]:
sol = ####

#### Plot your numerical solution 

In [ ]:
using Plots

a = heatmap(0:n_upperbound, 0:n_upperbound, sol(0),
    c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = 0 sec")

b = heatmap(0:n_upperbound, 0:n_upperbound, sol(T/16), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/16) .*" sec")

c = heatmap(0:n_upperbound, 0:n_upperbound, sol(T/8), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/8) .*" sec")

d = heatmap(0:n_upperbound, 0:n_upperbound, sol(T/4), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/4) .*" sec")

e = heatmap(0:n_upperbound, 0:n_upperbound, sol(T/2), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/2) .*" sec")

f = heatmap(0:n_upperbound, 0:n_upperbound, sol(T), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T) .*" sec")

layout = @layout [a b c 
                d e f]
Plots.plot(a, b, c, d, e, f; layout = layout,  colorbar = false, size = (1400, 900),  margin = 30Plots.px)

#### How about draws with the Gillespie algorithm?

In [ ]:
using Distributions, StatsBase 

propensities = function(κ, x, ν, Nᵣ)
    ν_eval = zeros(Nᵣ)
    for r in 1:Nᵣ
        ν_eval[r] = ν[r](κ[r], x) 
    end
    return ν_eval
end

gillespie_alg = function(SM, κ, ν, x₀, stoptime)
    t = 0.0
    x = x₀
    
    Nₛ, Nᵣ = size(SM)
    
    tt = [t]
    xx = copy(x)     
    
    k=1
    
    while (t <= stoptime && k<1e6)
        
        # step 1
        ν_eval = propensities(κ, x, ν, Nᵣ)              
        α = sum(ν_eval) 

        # step 2
        τ = rand(Exponential(1/α))                                   
        t += τ

        # step 3
        index_j = StatsBase.sample(1:Nᵣ, Weights(ν_eval./α))
        
        x = x + SM[:, index_j]
        append!(tt, t)
        xx = cat(xx, x, dims=2)
        
        k+=1
    end
    
    return tt, xx'
end

In [ ]:
# Compute the stoichiometric matrix 
Products = ####
Reactants = ####
SM = (Products-Reactants)'

# Compute the propensity function for each of the reactions
ν₁ =  function(κ₁, x) return #### end       
ν₂ = function(κ₂, x) return #### end   
ν₃ = function(κ₃, x) return #### end   
ν₄ = function(κ₄, x) return #### end   

x₀ = [0, 0]

N = 1e4                                                  # number of realisations
pp = zeros(n_upperbound+1, n_upperbound+1)               # matrix of probabilities

for n in 1:N   
    tt, xx = gillespie_alg(SM, [κ₁, κ₂, κ₃, κ₄], [ν₁, ν₂, ν₃, ν₄], x₀, T)
  
     nA, nB = xx[end, :]    

     if (nA <= n_upperbound && nB <= n_upperbound) 
        pp[nA+1, nB+1] += 1
     end
end

pp /= N;

In [ ]:
a = heatmap(0:n_upperbound, 0:n_upperbound, sol(T),
    c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="Solution to the \nChemical Master equation @ t = 20sec")

b = heatmap(0:n_upperbound, 0:n_upperbound, pp, c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="Empirical density with the \nGillespie algorithm @ t = 20sec")

layout = @layout [a b]
Plots.plot(a, b; layout = layout,  colorbar = false, size = (800, 400),  margin = 30Plots.px)

## 2. System 2

$\mathcal{A}\xrightarrow{\kappa_1}\mathcal{0}$

$\mathcal{B}\xrightarrow{\kappa_2}\mathcal{A}$

$\emptyset \xrightarrow{\kappa_3}\mathcal{B}$

$\mathcal{A}+\mathcal{B}\xrightarrow{\kappa_4}2\mathcal{A}$

$\mathcal{A}+\mathcal{B}\xrightarrow{\kappa_5}2\mathcal{B}$

$\mathcal{A}\xrightarrow{\kappa_6}\emptyset$

$\mathcal{B}\xrightarrow{\kappa_7}\emptyset$

#### Define a function that computes the RHS of the chemical master equation.

In [ ]:
n_upperbound = 20

## Again, if im matrix form:
Avec = collect(0:n_upperbound)
Bvec = collect(0:n_upperbound)

p = Iterators.product(Avec, Bvec)
points = collect.(p)
points_reshaped = reduce(hcat,points)
Apoints_matrix = reshape(points_reshaped[1,:], n_upperbound+1, n_upperbound+1)
Bpoints_matrix = reshape(points_reshaped[2,:], n_upperbound+1, n_upperbound+1)

function AB_system!(du, u, p, t)
    
    κ₁, κ₂, κ₃, κ₄, κ₅, κ₆, κ₇, V = p

    u ./= sum(u) 

    l1, l2 = size(u)
    u_padded = [zeros(l1) u zeros(l1)]
    u_padded = ([zeros(l2+2) u_padded' zeros(l2+2)])'
   
    A = (-κ₁*####
         -κ₂*####
         -κ₃*#### 
         -κ₄*####  
         -κ₅*####        
         -κ₆*####
         -κ₇*####)
    B = κ₁*####   
    C = κ₂*####
    D = κ₃*####
    E = κ₄*####
    F = κ₅*####
    G = κ₆*####
    H = κ₇*####

    du .=  (A .* #### 
           +B .* #### 
           +C .* #### 
           +D .* #### 
           +E .* ####
           +F .* ####
           +G .* ####
           +H .* ####)
end

#### Set the initial condition, the time interval of interest and the parameters.

In [ ]:
u₀ = zeros(####, ####)
u₀[6, 1] = ####

T = 20.0
tspan = ####

κ₁, κ₂, κ₃, κ₄, κ₅, κ₆, κ₇, V =  ####
p = [κ₁, κ₂, κ₃, κ₄, κ₅, κ₆, κ₇, V];

#### Create an ordinary differential equation problem with the variables defined above.

In [ ]:
prob = ####

#### Solve the ODE problem

You can for example solve the problem using Tsit5()

In [ ]:
sol = ####

#### Plot your numerical solution 

In [ ]:
a = heatmap(0:n_upperbound, 0:n_upperbound, sol(0),
    c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = 0 sec")

b = heatmap(0:n_upperbound,
    0:n_upperbound, sol(T/16), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/16) .*" sec")

c = heatmap(0:n_upperbound,
    0:n_upperbound, sol(T/8), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/8) .*" sec")

d = heatmap(0:n_upperbound,
    0:n_upperbound, sol(T/4), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/4) .*" sec")

e = heatmap(0:n_upperbound,
    0:n_upperbound, sol(T/2), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T/2) .*" sec")

f = heatmap(0:n_upperbound,
    0:n_upperbound, sol(T), c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="t = ".* string(T) .*" sec")

layout = @layout [a b c 
                  d e f]
Plots.plot(a, b, c, d, e, f;layout = layout,  colorbar = false, size = (1400, 900),  margin = 30Plots.px)

#### How about draws with the Gillespie algorithm?

In [ ]:
# Compute the stoichiometric matrix 
Products = #### 
Reactants = #### 
SM = (Products-Reactants)'

# Compute the propensity function for each of the reactions
ν₁ =  function(κ₁, x) return ####  end       
ν₂ = function(κ₂, x) return #### end   
ν₃ = function(κ₃, x) return #### end   
ν₄ = function(κ₄, x) return ####  end
ν₅ = function(κ₅, x) return #### end
ν₆ = function(κ₆, x) return #### end
ν₇ = function(κ₇, x) return #### end
 
x₀ = ####

N = 1e4                                                  # number of realisations
pp = zeros(n_upperbound+1, n_upperbound+1)               # vector of probabilities

for n in 1:N   
     tt, xx = gillespie_alg(SM, [κ₁, κ₂, κ₃, κ₄, κ₅, κ₆, κ₇], [ν₁, ν₂, ν₃, ν₄, ν₅,ν₆, ν₇], x₀, T)
  
     nA, nB = xx[end, :]    

     if (nA <= n_upperbound && nB <= n_upperbound) 
        pp[nA+1, nB+1] += 1
     end
end

pp /= N;

In [ ]:
a = heatmap(0:n_upperbound, 0:n_upperbound, sol(T),
    c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="Solution to the \nChemical Master equation @ t = 20sec")

b = heatmap(0:n_upperbound, 0:n_upperbound, pp, c=:coolwarm,
    xlabel="# molecules of B", ylabel="# molecules of A", title="Empirical density with the \nGillespie algorithm @ t = 20sec")

layout = @layout [a b]
Plots.plot(a, b; layout = layout,  colorbar = false, size = (800, 400),  margin = 30Plots.px)

## Part 2: deterministic versus stochastic modelling

### Lotka-Volterra

Deterministic model:

$$d\begin{bmatrix}y_1(t) \\ y_2(t) \end{bmatrix} = \begin{bmatrix} \kappa_1 y_1(t)- \frac{\kappa_2}{V} y_1(t)y_2(t) \\ \frac{\kappa_2}{V} y_1(t)y_2(t)-\kappa_3y_2(t)  \end{bmatrix}dt, \quad \begin{bmatrix}y_1(0) \\ y_2(0) \end{bmatrix} = \begin{bmatrix}100 \\ 100 \end{bmatrix}.$$

Here, $y = (y_1, y_2)^\top$ and $\kappa_1 = 1$, $\kappa_2 =  5\times 10^{-3}$, and $\kappa_3 = 0.6$, and $V = 1$.

In [ ]:
function lotka_volterra!(du, u, p, t)
    κ₁, κ₂, κ₃, V  = p
    du .= [####,
           ####]
end

#### Set the initial condition, the time interval of interest and the parameters

In [ ]:
u₀ = ####

T = 15.0
tspan = ####

κ₁, κ₂, κ₃, V = ####
p = [κ₁, κ₂, κ₃, V];

#### Create an ordinary differential equation problem with the variables defined above.

In [ ]:
prob = ####

#### Solve the ODE problem

In [ ]:
sol = ####
# OR sol2 = solve(prob2, #### (algorithm name); #### (parameters))

#### How about draws with the Gillespie algorithm?

In [ ]:
Products = ####
Reactants = ####
SM = (Products-Reactants)'

ν₁ =  function(κ₁, y) return #### end       
ν₂ = function(κ₂, y) return #### end          
ν₃ = function(κ₃, y) return #### end   

y₀ = u₀

In [ ]:
tt, yy = gillespie_alg(SM, [κ₁, κ₂, κ₃], [ν₁, ν₂, ν₃], y₀, T)

In [ ]:
plot(sol, linewidth= 6.0, alpha=0.5, colour=[:darkblue :darkorange], size=(1200,400), linestyle=:dash, labels = ["y1(t)" "y2(t)"],
    xlabel="time", ylabel="# individuals", title="Lotka-Volterra model: 1 realisation vs ODE solution") 
plot!(tt, yy, linetype=:steppre, linewidth = 2.0, alpha=1.0, legend=:topleft, labels = ["realisation of Y1(t)" "realisation of Y2(t)"], colour=[:darkblue :darkorange])

#### Stochastic mean vs ODE solution

In [ ]:
left_bin_edges = 0.0:0.001:T                 # collection of times     
mm = zeros(length(left_bin_edges), 2)        # vector to store stochastic mean

Nₛ = 200                                     # number of realisations

for nₛ in 1:Nₛ    
    tt, yy = gillespie_alg(SM, [κ₁, κ₂, κ₃], [ν₁, ν₂, ν₃], y₀, T)
    
    ## Computing stochastic mean (code not optimized yet)
    ## inputs: left_bin_edges, tt, yy
    ## output: yy_extended (binned version of yy)
        idxs = searchsortedfirst.(Ref(left_bin_edges[1:end-1]), tt)

        for j in 1:length(idxs)
            if j == 1
                yy_extended = repeat(yy[j,:]', idxs[j])
            else
                updates = repeat(yy[j,:]', idxs[j]-idxs[j-1])
                if size(updates, 1) > 0
                   yy_extended = [yy_extended; updates] 
                else
                    yy_extended[end, :] += yy[j,:]
                end
            end
        end
        ## Filling the end section
        if idxs[end] < length(left_bin_edges) 
            yy_extended = [yy_extended; repeat(yy[end, :]', length(left_bin_edges) -  size(yy_extended, 1))]
        end

    mm += yy_extended./Nₛ
end

In [ ]:
plot(sol, linewidth= 6.0, alpha=0.5, colour=[:darkblue :darkorange], size=(1200,400), linestyle=:dash, labels = ["y1(t)" "y2(t)"],
        xlabel="time", ylabel="# individuals", title="Lotka-Volterra model: Estimate for the stochastic mean vs ODE solution") 
plot!(left_bin_edges, mm, linetype=:steppre, linewidth = 2.0, alpha=1.0, legend=:bottomright,  labels = ["E(Y1(t)] estimate" "E[Y2(t)] estimate"], colour=[:darkblue :darkorange])

## Homework